In [1]:
import pandas as pd
import numpy as np

import gensim
from gensim.utils import simple_preprocess
import nltk
from nltk.corpus import stopwords

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split

import tensorflow

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
dataset = pd.read_csv('/content/drive/MyDrive/Predicting Effective Arguments/Datasets/train.csv')
dataset.head()

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",Position,Adequate
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,Claim,Adequate
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",Evidence,Adequate
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,Counterclaim,Adequate


In [4]:
nltk.download('stopwords')
sw = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
def clean_data(sentence):
  tokens = simple_preprocess(sentence)
  tokens = [token for token in tokens if not token in sw]
  return ' '.join(tokens)
  #return tokens

In [6]:
dataset['Tokens'] = dataset['discourse_text'].apply(clean_data)
dataset.head()

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness,Tokens
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate,hi isaac going writing face mars natural landf...
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",Position,Adequate,perspective think face natural landform dont t...
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,Claim,Adequate,think face natural landform life mars descover...
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",Evidence,Adequate,life mars would know reason think natural land...
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,Counterclaim,Adequate,people thought face formed alieans thought lif...


In [7]:
dataset.drop(['discourse_id', 'essay_id'], axis=1, inplace=True)

In [8]:
dataset.head()

,discourse_text,discourse_type,discourse_effectiveness,Tokens
0,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate,hi isaac going writing face mars natural landf...
1,"On my perspective, I think that the face is a ...",Position,Adequate,perspective think face natural landform dont t...
2,I think that the face is a natural landform be...,Claim,Adequate,think face natural landform life mars descover...
3,"If life was on Mars, we would know by now. The...",Evidence,Adequate,life mars would know reason think natural land...
4,People thought that the face was formed by ali...,Counterclaim,Adequate,people thought face formed alieans thought lif...


In [9]:
max_count = max(len(sentence.split(' ')) for sentence in dataset['Tokens'])
max_count

373

# Encoding

In [10]:
# import numpy as np
# from tensorflow.keras.utils import to_categorical, plot_model
# from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
# from tensorflow.keras.preprocessing.sequence import pad_sequences

# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense
# from tensorflow.keras.layers import LSTM, Bidirectional
# from tensorflow.keras.layers import Embedding

In [11]:
# # integer encode sequences of words
# tokenizer = Tokenizer()
# tokenizer.fit_on_texts(dataset['Tokens'])
# #sequences = tokenizer.texts_to_sequences(dataset['Tokens'])

In [12]:
# def encode(sentence):
#   seq = tokenizer.texts_to_sequences([sentence])[0]
#   encoded = pad_sequences([seq], maxlen=max_count, truncating='pre')
#   return encoded[0]

In [13]:
# dataset['Encoded'] = dataset['Tokens'].apply(encode)
# dataset.head()

In [14]:
ohe = OneHotEncoder()

In [15]:
discourse_type_ohe = pd.DataFrame(ohe.fit_transform(dataset[['discourse_type']]).toarray())
discourse_type_ohe.head()

,0,1,2,3,4,5,6
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [16]:
dataset = pd.concat([dataset['Tokens'], discourse_type_ohe, dataset['discourse_effectiveness']], axis=1)
dataset.head()

,Tokens,0,1,2,3,4,5,6,discourse_effectiveness
0,hi isaac going writing face mars natural landf...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,Adequate
1,perspective think face natural landform dont t...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Adequate
2,think face natural landform life mars descover...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Adequate
3,life mars would know reason think natural land...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Adequate
4,people thought face formed alieans thought lif...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,Adequate


# LSTM

In [17]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
import tensorflow

In [18]:
dataset.head()

,Tokens,0,1,2,3,4,5,6,discourse_effectiveness
0,hi isaac going writing face mars natural landf...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,Adequate
1,perspective think face natural landform dont t...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Adequate
2,think face natural landform life mars descover...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Adequate
3,life mars would know reason think natural land...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Adequate
4,people thought face formed alieans thought lif...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,Adequate


In [19]:
voc_size = 16000
embedding_vector_features = 500

In [20]:
# One hot representation of words
onehot_repr=[one_hot(sentence,voc_size) for sentence in dataset['Tokens']] 
#onehot_repr

In [21]:
onehot_repr

[[10464,
  13991,
  9231,
  6965,
  11055,
  13794,
  9385,
  8136,
  12937,
  13794,
  11879,
  944,
  8366,
  5061,
  923,
  13794,
  11055,
  11398,
  12149,
  8366,
  10068,
  8136,
  11901,
  12937,
  13794,
  9385,
  8136],
 [12705,
  15657,
  11055,
  9385,
  8136,
  8519,
  15657,
  12937,
  13794,
  10706,
  11225,
  3261,
  15657,
  9385,
  8136],
 [15657, 11055, 9385, 8136, 12937, 13794, 12914, 9247],
 [12937,
  13794,
  1911,
  10068,
  15732,
  15657,
  9385,
  8136,
  1069,
  3740,
  13794,
  4792,
  15745,
  11686,
  14725,
  8425,
  931,
  9682,
  15782,
  12522,
  931,
  10068,
  9385,
  8136,
  1042,
  12522,
  2898,
  9385,
  8136],
 [41, 9703, 11055, 939, 2344, 9703, 12937, 13794],
 [11039, 12050, 12937, 13794, 13932, 15657, 12937, 13794],
 [14725,
  8425,
  4612,
  13794,
  11078,
  4161,
  7254,
  15782,
  11091,
  414,
  820,
  6777,
  5061,
  923,
  13794,
  1393,
  5049,
  11055,
  9385,
  8136],
 [13617,
  9703,
  11879,
  2344,
  3634,
  11039,
  4637,
  1565

In [22]:
one_hot_dict = dict()

In [23]:
for encodingList, tokensList in zip(onehot_repr, dataset['Tokens']):
  tokensList = tokensList.split(' ')
  for index, word in zip(encodingList, tokensList):
    one_hot_dict[word] = index

In [24]:
# Embedding the sentences
embedded_docs = pad_sequences(onehot_repr,padding='pre',maxlen=max_count)
print(embedded_docs)

[[    0     0     0 ... 13794  9385  8136]
 [    0     0     0 ... 15657  9385  8136]
 [    0     0     0 ... 13794 12914  9247]
 ...
 [    0     0     0 ... 13079  7588 12050]
 [    0     0     0 ... 12504  7417 14508]
 [    0     0     0 ...  5632 15052 11667]]


In [25]:
discourse_type_ohe = np.array(discourse_type_ohe)
discourse_type_ohe

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [26]:
embedded_docs.shape

(36765, 373)

In [27]:
embedded_docs = np.hstack((embedded_docs, discourse_type_ohe))
embedded_docs

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [28]:
embedded_docs.shape

(36765, 380)

In [29]:
sum(pd.DataFrame(embedded_docs).isnull().sum())

0

In [30]:
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=embedded_docs.shape[1]))
model.add(Dropout(0.3))

model.add(LSTM(512, activation='relu', return_sequences=True))
model.add(Dropout(0.3))

model.add(LSTM(256, activation='relu', return_sequences=True))
model.add(Dropout(0.3))

model.add(LSTM(128, activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(128,activation='relu'))
model.add(Dense(3,activation='softmax'))
optimizer = tensorflow.keras.optimizers.Adadelta(learning_rate=.01)
model.compile(loss= 'categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [31]:
my_callbacks_lstm = [
    EarlyStopping(patience=2),
    ModelCheckpoint(filepath='/content/drive/MyDrive/Predicting Effective Arguments/LSTM_WordEmbeddings/Callbacks/model.{epoch:02d}-{val_loss:.2f}.h5'),
    TensorBoard(log_dir='/content/drive/MyDrive/Predicting Effective Arguments/LSTM_WordEmbeddings/logs'),
]

In [32]:
le = LabelEncoder()
dataset['le_discourse_effectiveness'] = le.fit_transform(dataset['discourse_effectiveness'])
dataset.head()

,Tokens,0,1,2,3,4,5,6,discourse_effectiveness,le_discourse_effectiveness
0,hi isaac going writing face mars natural landf...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,Adequate,0
1,perspective think face natural landform dont t...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Adequate,0
2,think face natural landform life mars descover...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Adequate,0
3,life mars would know reason think natural land...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Adequate,0
4,people thought face formed alieans thought lif...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,Adequate,0


In [33]:
ohe_discourse_effectiveness = pd.get_dummies(dataset['discourse_effectiveness'])
ohe_discourse_effectiveness

,Adequate,Effective,Ineffective
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0
...,...,...,...
36760,1,0,0
36761,1,0,0
36762,1,0,0
36763,0,0,1


In [34]:
X_LSTM = embedded_docs.copy()
y_LSTM = ohe_discourse_effectiveness.copy()
#y_LSTM = dataset['le_discourse_effectiveness']

In [35]:
X_LSTM.shape, y_LSTM.shape

((36765, 380), (36765, 3))

In [36]:
X_train_lstm, X_test_lstm, y_train_lstm, y_test_lstm = train_test_split(X_LSTM, y_LSTM, test_size=.33, random_state=42, shuffle=True)

In [37]:
model.fit(X_train_lstm, y_train_lstm, validation_data=(X_test_lstm, y_test_lstm), epochs=10, batch_size=512, callbacks=my_callbacks_lstm)

Epoch 1/10
49/49 [==============================] - 138s 3s/step - loss: 1.0973 - accuracy: 0.3869 - val_loss: 1.0950 - val_accuracy: 0.5730
Epoch 2/10
49/49 [==============================] - 134s 3s/step - loss: 1.0931 - accuracy: 0.5678 - val_loss: 1.0891 - val_accuracy: 0.5730
Epoch 3/10
49/49 [==============================] - 132s 3s/step - loss: 1.0867 - accuracy: 0.5694 - val_loss: 1.0817 - val_accuracy: 0.5730
Epoch 4/10
49/49 [==============================] - 130s 3s/step - loss: 1.0795 - accuracy: 0.5694 - val_loss: 1.0733 - val_accuracy: 0.5730
Epoch 5/10
49/49 [==============================] - 131s 3s/step - loss: 1.0711 - accuracy: 0.5694 - val_loss: 1.0639 - val_accuracy: 0.5730
Epoch 6/10
49/49 [==============================] - 131s 3s/step - loss: 1.0617 - accuracy: 0.5694 - val_loss: 1.0536 - val_accuracy: 0.5730
Epoch 7/10
49/49 [==============================] - 123s 3s/step - loss: 1.0515 - accuracy: 0.5694 - val_loss: 1.0423 - val_accuracy: 0.5730
Epoch 8/10
49

# Check model's performance

In [38]:
test = pd.read_csv('/content/drive/MyDrive/Predicting Effective Arguments/Datasets/test.csv')
test.head()

,discourse_id,essay_id,discourse_text,discourse_type
0,a261b6e14276,D72CB1C11673,Making choices in life can be very difficult. ...,Lead
1,5a88900e7dc1,D72CB1C11673,Seeking multiple opinions can help a person ma...,Position
2,9790d835736b,D72CB1C11673,it can decrease stress levels,Claim
3,75ce6d68b67b,D72CB1C11673,a great chance to learn something new,Claim
4,93578d946723,D72CB1C11673,can be very helpful and beneficial.,Claim


In [39]:
test['Tokens'] = test['discourse_text'].apply(clean_data)
test.head()

,discourse_id,essay_id,discourse_text,discourse_type,Tokens
0,a261b6e14276,D72CB1C11673,Making choices in life can be very difficult. ...,Lead,making choices life difficult people often ask...
1,5a88900e7dc1,D72CB1C11673,Seeking multiple opinions can help a person ma...,Position,seeking multiple opinions help person make bet...
2,9790d835736b,D72CB1C11673,it can decrease stress levels,Claim,decrease stress levels
3,75ce6d68b67b,D72CB1C11673,a great chance to learn something new,Claim,great chance learn something new
4,93578d946723,D72CB1C11673,can be very helpful and beneficial.,Claim,helpful beneficial


In [40]:
test.drop(['discourse_id', 'essay_id'], axis=1, inplace=True)

In [41]:
discourse_type_ohe_test = pd.DataFrame(ohe.transform(test[['discourse_type']]).toarray())
discourse_type_ohe_test.head()

,0,1,2,3,4,5,6
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
def encode_test_tokens(sentence):
  encoded_tokens = [one_hot_dict[token] for token in sentence.split(' ')]
  return encoded_tokens

In [43]:
# One hot representation of words
one_hot_repr_test = [encode_test_tokens(sentence) for sentence in test['Tokens']]
#one_hot_repr_test


# Embedding the sentences
embedded_docs_test = pad_sequences(one_hot_repr_test, padding='pre', maxlen=max_count)
#print(embedded_docs_test)

discourse_type_ohe_test = np.array(discourse_type_ohe_test)
#discourse_type_ohe

embedded_docs_test = np.hstack((embedded_docs_test, discourse_type_ohe_test))
embedded_docs_test

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [44]:
sum(pd.DataFrame(embedded_docs_test).isnull().sum())

0

In [45]:
# test = pd.concat([test['Tokens'], discourse_type_ohe_test], axis=1)
# test.head()

In [46]:
# Loading the model
#model = tensorflow.keras.models.load_model('/content/drive/MyDrive/Predicting Effective Arguments/LSTM_WordEmbeddings/Callbacks/model.02-nan.h5')

In [47]:
embedded_docs.shape, embedded_docs_test.shape

((36765, 380), (10, 380))

In [48]:
model.predict(embedded_docs_test)

array([[0.4471385 , 0.3088151 , 0.24404636],
       [0.45085824, 0.3080052 , 0.2411365 ],
       [0.45442858, 0.3069843 , 0.23858708],
       [0.45270708, 0.30752885, 0.23976415],
       [0.4552722 , 0.30669835, 0.2380294 ],
       [0.44653726, 0.30906707, 0.24439564],
       [0.44691652, 0.30873287, 0.24435058],
       [0.45246175, 0.3072596 , 0.24027877],
       [0.4470087 , 0.30903032, 0.24396095],
       [0.4469553 , 0.3089061 , 0.24413858]], dtype=float32)